In [1]:
import spacy 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt 
import spacy
from collections import Counter 
import json
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

In [3]:
submissions1 = pd.read_json("/Users/ritikabrahmadesam/Downloads/opiates_submissions1.json")
submissions2 = pd.read_json("/Users/ritikabrahmadesam/Downloads/opiates_submissions2.json")

### This analysis aims to track the frequencies of certain words over time

In [5]:
submissions = pd.concat([submissions1, submissions2])

In [8]:
#create a column with real datetimes
submissions['created_utc'] = submissions['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date))
submissions = submissions.sort_values(by='created_utc', ascending=True)

In [10]:
#remove the stop words and punctionationfrom a line of text
def remove_stops(text):
    nlp = spacy.load("en_core_web_sm")
    words = [word.lower() for word in text.split() if word.lower() not in sw_spacy]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    new_text = " ".join(words)
    for elem in punc:
        new_text = new_text.replace(elem, "")
    return new_text

In [11]:
df_2011 = submissions[submissions['created_utc'].dt.year==2011]

In [14]:
df_2011 = submissions[submissions['created_utc'].dt.year==2012]
curr_df = {"january": df_2011[df_2011['created_utc'].dt.month==1],
           "febuary": df_2011[df_2011['created_utc'].dt.month==2],
           "march": df_2011[df_2011['created_utc'].dt.month==3],
            "april": df_2011[df_2011['created_utc'].dt.month==4],
            "may": df_2011[df_2011['created_utc'].dt.month==5],
            "june": df_2011[df_2011['created_utc'].dt.month==6],
            "august": df_2011[df_2011['created_utc'].dt.month==8],
            "september": df_2011[df_2011['created_utc'].dt.month==9],
            "october": df_2011[df_2011['created_utc'].dt.month==10],
            "november": df_2011[df_2011['created_utc'].dt.month==11],
            "december": df_2011[df_2011['created_utc'].dt.month==12]}

In [15]:
#clean the text by removing stop words, punctuation, and empty entries
for month in curr_df.keys():
    curr_month = curr_df[month]
    text = curr_month['selftext'].to_list()
    text = [remove_stops(t) for t in text]
    while("" in text):
        text.remove("")
    text = " ".join(text)
    text = text.split()
    frequent = Counter(text) 
    most_occur = frequent.most_common(50) 
    json_format = json.dumps(dict(most_occur))
    with open(month + ".json", "w") as outfile:
        outfile.write(json_format)